In [13]:
# PNC

import pickle
import numpy as np

pncdir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/PNC/'

pncdemo = pickle.load(open(f'{pncdir}/demographics.pkl', 'rb'))

task = 'nback'

aamale = []
aafemale = []
eamale = []
eafemale = []

for sub in pncdemo['Race']:
    if sub not in pncdemo['Sex']:
        continue
    a = pncdemo['age_at_cnb'][sub]
    r = pncdemo['Race'][sub]
    s = pncdemo['Sex'][sub]
    if r not in ['AA', 'EA']:
        continue
    try:
        p = np.load(f'{pncdir}/fc/{sub}_task-{task}_fc.npy')
    except:
        continue
    if s == 'M' and r == 'AA':
        aamale.append(p)
    elif s == 'M' and r == 'EA':
        eamale.append(p)
    elif s == 'F' and r == 'AA':
        aafemale.append(p)
    elif s == 'F' and r == 'EA':
        eafemale.append(p)

aamale = np.stack(aamale)
aafemale = np.stack(aafemale)
eamale = np.stack(eamale)
eafemale = np.stack(eafemale)

print(len(aamale), len(aafemale), len(eamale), len(eafemale))

266 353 342 329


In [1]:
# BSNIP

import pickle
import numpy as np

bsnipdir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/BSNIP/'

bsnipdemo = pickle.load(open(f'{bsnipdir}/demographics.pkl', 'rb'))

aamale = []
aafemale = []
eamale = []
eafemale = []

for sub in bsnipdemo['Race']:
    if sub not in bsnipdemo['sex']:
        continue
    try:
        dx = bsnipdemo['DXGROUP_1'][sub]
        if dx != 'NC':
            continue
    except:
        continue
    r = bsnipdemo['Race'][sub]
    s = bsnipdemo['sex'][sub]
    if r not in ['AA', 'CA']:
        continue
    try:
        p = np.load(f'{bsnipdir}/fc/{sub}_task-unk_fc.npy')
    except:
        continue
    if s == 's1.0' and r == 'AA':
        aamale.append(p)
    elif s == 's1.0' and r == 'CA':
        eamale.append(p)
    elif s == 's2.0' and r == 'AA':
        aafemale.append(p)
    elif s == 's2.0' and r == 'CA':
        eafemale.append(p)

aamale = np.stack(aamale)
aafemale = np.stack(aafemale)
eamale = np.stack(eamale)
eafemale = np.stack(eafemale)

print(len(aamale), len(aafemale), len(eamale), len(eafemale))

28 42 63 87


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

def choose(arr, n):
    idcs = np.random.permutation(len(arr))
    return arr[idcs[:n]]

accs = []
N = 250

for _ in range(10):
    xtr = np.concatenate([choose(aamale, N), choose(eafemale, N)])
    xt = np.concatenate([choose(eamale, N), choose(aafemale, N)])
    ytr = np.concatenate([np.zeros(N), np.ones(N)])
    yt = ytr
    
    # x = np.concatenate([choose(aamale, N), choose(aafemale, N), choose(eamale, N), choose(eafemale, N)])
    # y = np.concatenate([np.zeros(2*N), np.ones(2*N)])
    # xtr, xt, ytr, yt = train_test_split(x, y, stratify=y, train_size=2*N)

    reg = LogisticRegression(max_iter=1000).fit(xtr, ytr)
    yhat = reg.predict(xt)
    acc = np.mean(yhat == yt)
    print(acc)
    accs.append(acc)

print(np.mean(accs), np.std(accs))

0.27
0.28
0.318
0.302
0.282
0.308
0.278
0.306
0.296
0.292
0.29319999999999996 0.014675149062275308
